In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(df_market_data
        [['price_change_percentage_24h','price_change_percentage_7d',
          'price_change_percentage_14d','price_change_percentage_30d',
          'price_change_percentage_60d','price_change_percentage_200d',
          'price_change_percentage_1y']])

In [ ]:
# Create a DataFrame with the scaled data
market_scaled_df = pd.DataFrame(df_market_data_scaled,
                                columns=['price_change_percentage_24h','price_change_percentage_7d',
                                    'price_change_percentage_14d','price_change_percentage_30d',
                                    'price_change_percentage_60d','price_change_percentage_200d',
                                    'price_change_percentage_1y'])

# Copy the crypto names from the original data
market_scaled_df['Crypto_names']=df_market_data.index

# Set the coinid column as index
market_scaled_df=market_scaled_df.set_index("Crypto_names")

# Display sample data
market_scaled_df.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k":k,
    "inertia":inertia
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)
elbow_df.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)
elbow_plot

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:4**

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)


In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_scaled_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
market_predictions = model.predict(market_scaled_df)

# Print the resulting array of cluster values.
market_predictions

In [ ]:
# Create a copy of the DataFrame
market_scaled_prediction_df = market_scaled_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_scaled_prediction_df['Prediction']=market_predictions

# Display sample data
market_scaled_prediction_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_scaled_plot = market_scaled_prediction_df.hvplot.scatter(
                    x="price_change_percentage_24h",
                    y="price_change_percentage_7d",
                    by="Prediction",
                    hover_cols=["Crypto_names"]
)


market_scaled_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_market = pca.fit_transform(df_market_data)
# View the first five rows of the DataFrame. 
pca_market[0:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

In [ ]:
explained_variance = np.sum(pca.explained_variance_ratio_)
explained_variance

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:0.999813916129899

In [ ]:
# Create a new DataFrame with the PCA data.
pca_market_df = pd.DataFrame(
    pca_market,
    columns=["PCA1","PCA2", "PCA3"]
)
# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
pca_market_df["Crypto_names"]=df_market_data.index

# Set the coinid column as index
pca_market_df=pca_market_df.set_index("Crypto_names")

# Display sample data
pca_market_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
pca_k = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in pca_k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(pca_market_df)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)
df_elbow_pca.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca
    


#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:2**


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:Yes, the best k value was initially 4 but using PCA the best value is now 2** 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=2, random_state=0)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(pca_market_df)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_predictions = model.predict(pca_market_df)

# Print the resulting array of cluster values.
pca_predictions

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_market_prediction_df = pca_market_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_market_prediction_df["PCA_Prediction"]=pca_predictions

# Display sample data
pca_market_prediction_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_market_plot=pca_market_prediction_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="PCA_Prediction",
    hover_cols=["Crypto_names"]
    
)

pca_market_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_plot + elbow_plot_pca

In [ ]:
# Composite plot to contrast the clusters
market_scaled_plot + pca_market_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:The utilization of PCA helped in reducing the cluster size which results in having a more accurate display** 